In [46]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from sklearn.linear_model import LinearRegression
import numpy as np


In [47]:

df = yf.download('AAPL', start='2020-01-01', end='2024-12-31')
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

/var/folders/0f/l_wv06ws6k3063l3r6mv6xb80000gn/T/ipykernel_33056/612593103.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2020-01-01', end='2024-12-31')
[*********************100%***********************]  1 of 1 completed


In [48]:
n_lags = 5

for lag in range(1, n_lags + 1):
    df[f'lag_{lag}'] = df['Close'].shift(lag)
df['ma_3'] = df['Close'].rolling(window=3).mean()
df['ma_5'] = df['Close'].rolling(window=5).mean()
df.dropna(inplace=True)

In [49]:


window = 20

X = df[[f'lag_{lag}' for lag in range(1, n_lags + 1)] + ['ma_3', 'ma_5']].values
y = df['Close'].shift(-1).values

preds = np.full_like(y, fill_value=np.nan, dtype=np.float64)

for i in range(window, len(df) - 1):
    X_train = X[i - window:i]
    y_train = y[i - window:i]

    model = LinearRegression().fit(X_train, y_train)
    preds[i] = model.predict(X[i].reshape(1, -1))[0]
    
df['pred_close'] = preds
df.dropna(inplace=True)


In [50]:


class LinearRegressionStrategy(Strategy):
    def init(self):
        self.predicted = df['pred_close'].values

    def next(self):
        i = len(self.data) - 1
        if self.predicted[i] > self.data.Close[-1]:
            if not self.position.is_long:
                self.buy()
        else:
            if self.position.is_long:
                self.position.close()

In [51]:
bt = Backtest(df, LinearRegressionStrategy, cash=1000)
stats = bt.run()
bt.plot(resample=False, superimpose=False)
print(stats)

Backtest.run:   0%|          | 0/1230 [00:00<?, ?bar/s]

/opt/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:701: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:701: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2020-02-07 00:00:00
End                       2024-12-27 00:00:00
Duration                   1785 days 00:00:00
Exposure Time [%]                    68.96832
Equity Final [$]                    923.17047
Equity Peak [$]                    1018.43893
Return [%]                           -7.68295
Buy & Hold Return [%]               228.73331
Return (Ann.) [%]                    -1.62317
Volatility (Ann.) [%]                21.21028
CAGR [%]                             -1.12224
Sharpe Ratio                         -0.07653
Sortino Ratio                        -0.10335
Calmar Ratio                         -0.04401
Alpha [%]                          -119.34902
Beta                                  0.48819
Max. Drawdown [%]                   -36.88084
Avg. Drawdown [%]                   -18.87095
Max. Drawdown Duration     1773 days 00:00:00
Avg. Drawdown Duration      890 days 00:00:00
# Trades                                  231
Win Rate [%]                      